# IE–IATA Datathon 2025 — EU27 SAF Scenario Model

This notebook implements a transparent, end-to-end workflow to:

1. Reconstruct historical EU27 aviation fuel demand using Eurostat energy data for air transport.
2. Project total fuel demand to 2050 using a simple growth assumption consistent with EUROCONTROL traffic outlooks.
3. Build two stylised SAF scenarios:
   - **Scenario 0 (S0):** Business-as-usual (slow SAF uptake).
   - **Scenario 1 (S1):** Policy-aligned scenario (ReFuelEU SAF blending floors + moderate acceleration).
4. Compute CO₂ emissions and avoided emissions under each scenario.
5. Produce the **official datathon output dataset**:  
   `Year, Scenario, Total_Fuel, SAF_Share, CO2_Emissions, Avoided_CO2`.
6. Add:
   - **Probability-weighted results** (reflecting the IEA warning that higher-warming paths are more likely without policy).
   - A **cost & sensitivity analysis** on SAF price premiums, ETS prices, and SAF lifecycle performance.

**Key data sources & references:**

- Eurostat energy data for aviation (EU27 final energy consumption in air transport).  
- EUROCONTROL *Aviation Outlook 2050* for long-term traffic and fuel demand trends.  
- European Commission *ReFuelEU Aviation* regulation for minimum SAF blending mandates.  
- IATA / ICAO / EASA publications for SAF cost ranges and lifecycle emission reductions.  
- IEA Net Zero and fossil fuels outlook for scenario probability intuition.


In [1]:
# ============================================
# STEP 0 — Imports & Global Config
# ============================================
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

plt.rcParams["figure.figsize"] = (8, 5)
plt.rcParams["axes.grid"] = True

# Base path (adapt if needed)
base_dir = Path().resolve().parent
raw_data_path = base_dir / "data" / "01-raw_data"


## 1. Historical EU27 fuel demand (1990–2022)

We start from the Eurostat dataset on **final energy consumption in air transport** for EU27. The steps are:

- Filter the **EU-27** time series from the raw file.
- Convert total aviation final energy into **tonnes of fuel (Mt)** using a standard energy–fuel conversion factor  
  (we assume **0.043 TJ per kilotonne of jet fuel**, which is consistent with typical jet fuel energy content).
- Aggregate domestic and international aviation into a single **`Total_Fuel_Mt`** time series.

This gives a consistent historical fuel demand baseline for EU27 that we later use to calibrate our SAF scenarios.

**Justification of assumptions:**

- Using Eurostat as the historical source ensures official, EU-harmonised energy statistics.
- The energy-to-fuel conversion factor is standard for jet kerosene and is widely used in IPCC / aviation fuel calculations.


In [ ]:
# ============================================
# Historical EU27 aviation fuel demand
# ============================================

energy_path = raw_data_path / "eu-27_final energy consumption for different fuels used in air transport.csv"

df_energy = pd.read_csv(
    energy_path,
    skiprows=7,
    header=0,
    sep=",",
    engine="python"
)

# Keep only EU-27 series columns
eu27_cols = [col for col in df_energy.columns if "EU" in col]

df_energy = (
    df_energy[eu27_cols]
    .drop("Timeseries_EU-27_x1", axis=1)
    .apply(pd.to_numeric, errors="coerce")
)

# Index as years: 1990–2022
df_energy.index = range(1990, 2023)
df_energy.index.name = "Year"

# Aggregate domestic + international, convert to Mt fuel
# Assumption: 0.043 TJ/kt → 1 / (0.043 * 1e6) to go from TJ to Mt fuel
df_baseline = pd.DataFrame(
    {
        "Total_Domestic_Mt": df_energy[[c for c in df_energy if "Domestic" in c]].sum(axis=1),
        "Total_International_Mt": df_energy[[c for c in df_energy if "International" in c]].sum(axis=1),
        "Total_Fuel_Mt": df_energy.sum(axis=1)
    },
    index=df_energy.index
) * (1 / (0.043 * 1e6))

df_baseline.index.name = "Year"
df_fuel_hist = df_baseline.reset_index()[["Year", "Total_Fuel_Mt"]]

print("Historical fuel demand:")
display(df_fuel_hist.head())
display(df_fuel_hist.tail())


Historical fuel demand:


,Year,Total_Fuel_Mt
0,1990,22.607036
1,1991,22.050996
2,1992,23.298807
3,1993,24.054035
4,1994,25.042143


,Year,Total_Fuel_Mt
28,2018,46.034323
29,2019,47.023255
30,2020,20.513992
31,2021,25.125299
32,2022,39.526779


## 2. Fuel demand projections (2024–2050)

To keep the approach transparent and aligned with the datathon guidelines, we use a **constant annual growth rate** of:

- **+1.6% per year** (AAGR),

which is consistent with the **EUROCONTROL base traffic scenario** for long-term growth in flights and associated fuel demand.

Steps:

- Use a baseline of **39 Mt** in the mid-2020s, broadly in line with EUROCONTROL and IATA aviation outlooks.
- Apply compound growth from **2024 to 2050** using this rate.
- Add a simple estimate for **2023** and combine with the historical fuel series.

This yields a unified **fuel demand series from 1990 to 2050**, which serves as the core quantity in our SAF adoption scenarios.

**Why this is reasonable:**

- EUROCONTROL’s long-term outlook shows moderate post-COVID recovery followed by steady growth, rather than explosive or zero growth.
- A constant 1.6% growth rate is deliberately simple yet realistic, and easy to explain in the context of a datathon.


In [ ]:
# ============================================
# Project fuel demand to 2050
# ============================================

GROWTH_RATE = 0.016        # +1.6% AAGR (EUROCONTROL base scenario)
START_YEAR = 2024
END_YEAR = 2050
BASELINE_FUEL_MT = 39.0    # Recommended baseline for mid-2020s

years = np.arange(START_YEAR, END_YEAR + 1)
df_projection = pd.DataFrame({"Year": years}).set_index("Year")

# Initialize
df_projection.loc[START_YEAR, "Total_Fuel_Mt"] = BASELINE_FUEL_MT

# Apply CAGR
for year in years[1:]:
    prev = df_projection.loc[year - 1, "Total_Fuel_Mt"]
    df_projection.loc[year, "Total_Fuel_Mt"] = prev * (1 + GROWTH_RATE)

print("Projected fuel demand (2024–2050):")
display(df_projection.head())
display(df_projection.tail())


Projected fuel demand (2024–2050):


,Total_Fuel_Mt
Year,
2024,39.000000
2025,39.624000
2026,40.257984
2027,40.902112
2028,41.556546


,Total_Fuel_Mt
Year,
2046,55.300134
2047,56.184936
2048,57.083895
2049,57.997237
2050,58.925193


In [4]:
# Add a simple 2023 estimate and combine historical + projections

df_fuel_hist_2023 = pd.concat(
    [
        df_fuel_hist,
        pd.DataFrame({"Year": [2023], "Total_Fuel_Mt": [38.7]})  # simple estimate
    ],
    ignore_index=True
)

df_fuel_proj = df_projection.reset_index()  # Year, Total_Fuel_Mt

df_fuel_full = (
    pd.concat([df_fuel_hist_2023, df_fuel_proj], ignore_index=True)
    .sort_values("Year")
    .reset_index(drop=True)
)

print("Unified fuel series 1990–2050:")
display(df_fuel_full.head())
display(df_fuel_full.tail())


Unified fuel series 1990–2050:


,Year,Total_Fuel_Mt
0,1990,22.607036
1,1991,22.050996
2,1992,23.298807
3,1993,24.054035
4,1994,25.042143


,Year,Total_Fuel_Mt
56,2046,55.300134
57,2047,56.184936
58,2048,57.083895
59,2049,57.997237
60,2050,58.925193


## 3. SAF scenarios (S0: BAU, S1: Policy)

We model two stylised SAF pathways:

### Scenario 0 — S0 (Business-as-usual)

- Represents **slow SAF adoption**, consistent with limited policy support.
- SAF share rises roughly from **2% in 2025** to **30% by 2050**.
- This reflects a world where SAF scales, but more slowly than climate-ambitious pathways.

### Scenario 1 — S1 (Policy / ReFuelEU-aligned)

- Based on the **ReFuelEU Aviation** regulation, which sets **minimum SAF blending mandates** for EU fuel suppliers.
- We approximate the “floors” as:
  - 2% in 2025  
  - 6% in 2030  
  - 20% in 2035  
  - 70% in 2050  
- We then add an additional **+10 percentage points** of SAF share by 2050 to reflect the possibility that the EU overshoots minimum targets under stronger climate ambition.

### Emissions modelling

For each year and scenario, we compute:

- **Total fuel demand (Mt)** from the previous step.
- **SAF share** (fraction of fuel).
- **CO₂ emissions** using:
  - Fossil jet emission factor: **3.15 tCO₂ per tonne fuel** (standard aviation value).
  - SAF lifecycle emission reduction: **75% lower than fossil jet**, consistent with the datathon guide recommendation of 70–80%.

We also compute a **0% SAF counterfactual** (all fossil) to define **avoided emissions**:

\[
\text{Avoided CO₂} = \text{CO₂ with 0% SAF} - \text{CO₂ with modeled SAF share}
\]

**Justification:**

- S0 vs S1 follows the mentor’s suggested structure: a BAU path versus a policy-driven path.
- ReFuelEU blending floors provide a concrete policy anchor for minimum SAF shares.
- A 75% lifecycle reduction is the midpoint of the commonly cited 70–80% SAF LCA benefit band (IATA / ICAO / EASA).


In [ ]:
# ============================================
# SAF scenarios: S0 (BAU) & S1 (Policy)
# ============================================

EF_JET = 3.15              # tCO2 / tonne fossil fuel
SAF_LCA_REDUCTION = 0.75   # 75% life-cycle CO2 reduction vs fossil
EF_SAF = EF_JET * (1 - SAF_LCA_REDUCTION)

START_YEAR_MODEL = 2025
END_YEAR_MODEL = 2050

def saf_share_s0(year: int) -> float:
    """Scenario 0 (BAU): ~2% in 2025 to ~30% in 2050."""
    if year < 2025:
        return 0.01
    if year > 2050:
        year = 2050
    return 0.02 + (0.30 - 0.02) * (year - 2025) / (2050 - 2025)

# ReFuelEU minimum floors (approx)
refeuleu_floors = {2025: 0.02, 2030: 0.06, 2035: 0.20, 2050: 0.70}

def saf_floor_s1(year: int) -> float:
    years = sorted(refeuleu_floors.keys())
    if year <= years[0]:
        return refeuleu_floors[years[0]]
    if year >= years[-1]:
        return refeuleu_floors[years[-1]]
    for y0, y1 in zip(years[:-1], years[1:]):
        if y0 <= year <= y1:
            v0, v1 = refeuleu_floors[y0], refeuleu_floors[y1]
            return v0 + (v1 - v0) * (year - y0) / (y1 - y0)

def saf_share_s1(year: int) -> float:
    """Scenario 1 (Policy): at least ReFuelEU floors, +10pp by 2050."""
    floor = saf_floor_s1(year)
    extra = 0.10 * (year - 2025) / (2050 - 2025)  # +10 percentage points by 2050
    return min(1.0, floor + max(0, extra))


In [6]:
rows = []

for year in range(START_YEAR_MODEL, END_YEAR_MODEL + 1):
    row_fuel = df_fuel_full[df_fuel_full["Year"] == year]
    if row_fuel.empty:
        continue

    total_fuel_mt = float(row_fuel["Total_Fuel_Mt"].iloc[0])
    co2_baseline_mt = total_fuel_mt * EF_JET   # 0% SAF baseline

    for scenario in [0, 1]:
        if scenario == 0:
            saf_share = saf_share_s0(year)
        else:
            saf_share = saf_share_s1(year)

        saf_share = max(0.0, min(1.0, saf_share))
        jet_frac = 1 - saf_share
        saf_frac = saf_share

        co2_emissions_mt = total_fuel_mt * (
            jet_frac * EF_JET + saf_frac * EF_SAF
        )
        avoided_mt = co2_baseline_mt - co2_emissions_mt

        rows.append({
            "Year": year,
            "Scenario": scenario,
            "Total_Fuel_Mt": total_fuel_mt,
            "SAF_Share_frac": saf_share,
            "CO2_Emissions_Mt": co2_emissions_mt,
            "CO2_Baseline_no_SAF_Mt": co2_baseline_mt,
            "Avoided_CO2_Mt": avoided_mt,
        })

df_saf_model = pd.DataFrame(rows).sort_values(["Year", "Scenario"])

print("SAF scenario model (sample):")
display(df_saf_model.head())
display(df_saf_model.tail())


SAF scenario model (sample):


,Year,Scenario,Total_Fuel_Mt,SAF_Share_frac,CO2_Emissions_Mt,CO2_Baseline_no_SAF_Mt,Avoided_CO2_Mt
0,2025,0,39.624000,0.0200,122.943366,124.815600,1.872234
1,2025,1,39.624000,0.0200,122.943366,124.815600,1.872234
2,2026,0,40.257984,0.0312,123.845234,126.812650,2.967416
3,2026,1,40.257984,0.0320,123.769146,126.812650,3.043504
4,2027,0,40.902112,0.0424,124.744487,128.841652,4.097165


,Year,Scenario,Total_Fuel_Mt,SAF_Share_frac,CO2_Emissions_Mt,CO2_Baseline_no_SAF_Mt,Avoided_CO2_Mt
47,2048,1,57.083895,0.725333,81.995307,179.814269,97.818962
48,2049,0,57.997237,0.288800,143.120362,182.691297,39.570935
49,2049,1,57.997237,0.762667,78.191875,182.691297,104.499422
50,2050,0,58.925193,0.300000,143.851127,185.614358,41.763231
51,2050,1,58.925193,0.800000,74.245743,185.614358,111.368615


## 4. Official datathon output dataset

The datathon requires a single, auditable table with the following columns:

- `Year`
- `Scenario`  
  - 0 = Business-as-usual (BAU)  
  - 1 = Policy / ReFuelEU-aligned
- `Total_Fuel` (Mt)
- `SAF_Share` (% of total fuel)
- `CO2_Emissions` (Mt)
- `Avoided_CO2` (Mt, relative to a 0% SAF baseline with the same fuel demand)

In this step, we:

1. Format the scenario results into the required structure.
2. Export the final dataset as:

`ie_iata_output_dataset.csv`

This dataset is the core quantitative output that supports all charts, scenario comparisons, and policy discussions in our final presentation.


In [ ]:
# ============================================
# Datathon output dataset
# ============================================

df_output_final = df_saf_model.copy()
df_output_final["SAF_Share"] = df_output_final["SAF_Share_frac"] * 100

df_output_final = df_output_final[[
    "Year",
    "Scenario",
    "Total_Fuel_Mt",
    "SAF_Share",
    "CO2_Emissions_Mt",
    "Avoided_CO2_Mt"
]].rename(columns={
    "Total_Fuel_Mt": "Total_Fuel",
    "CO2_Emissions_Mt": "CO2_Emissions",
    "Avoided_CO2_Mt": "Avoided_CO2"
})

print("Final datathon output (sample):")
display(df_output_final.head())
display(df_output_final.tail())

df_output_final.to_csv("ie_iata_output_dataset.csv", index=False)


Final datathon output (sample):


,Year,Scenario,Total_Fuel,SAF_Share,CO2_Emissions,Avoided_CO2
0,2025,0,39.624000,2.00,122.943366,1.872234
1,2025,1,39.624000,2.00,122.943366,1.872234
2,2026,0,40.257984,3.12,123.845234,2.967416
3,2026,1,40.257984,3.20,123.769146,3.043504
4,2027,0,40.902112,4.24,124.744487,4.097165


,Year,Scenario,Total_Fuel,SAF_Share,CO2_Emissions,Avoided_CO2
47,2048,1,57.083895,72.533333,81.995307,97.818962
48,2049,0,57.997237,28.880000,143.120362,39.570935
49,2049,1,57.997237,76.266667,78.191875,104.499422
50,2050,0,58.925193,30.000000,143.851127,41.763231
51,2050,1,58.925193,80.000000,74.245743,111.368615


## 5. Probability-weighted results (IEA-inspired weighting)

The datathon mentor and the IEA have highlighted that, **without additional policy**, higher-warming pathways (i.e. weak climate action) are more likely.

To reflect this in a simple way, we assign:

- **70% probability** to **Scenario 0 (BAU / higher warming)**  
- **30% probability** to **Scenario 1 (Policy / lower warming)**

For each year we then compute:

- **Expected SAF share (%)**
- **Expected CO₂ emissions (Mt)**
- **Expected avoided emissions (Mt)**

by taking the probability-weighted average across S0 and S1.

This provides a **probability-weighted trajectory** that answers the question:

> “Given today’s policy mix, where do we realistically end up on SAF uptake and emissions?”

**Justification:**

- The IEA’s recent analysis suggests fossil fuel demand could keep rising for ~25 years without major new policies, implying that BAU/higher-warming paths are still more probable today.
- Using 70/30 is a transparent, easy-to-communicate choice that reflects this imbalance without pretending to know exact probabilities.


In [ ]:
# ============================================
# Probability-weighted expected outcomes
# (70% BAU, 30% Policy, based on IEA warning)
# ============================================

p_s0 = 0.7  # Scenario 0: BAU / higher warming
p_s1 = 0.3  # Scenario 1: Policy / lower warming
prob_map = {0: p_s0, 1: p_s1}

df_prob = df_saf_model.copy()
df_prob["Scenario_Prob"] = df_prob["Scenario"].map(prob_map)

rows = []
for year, grp in df_prob.groupby("Year"):
    total_fuel = grp["Total_Fuel_Mt"].iloc[0]
    exp_saf_share = (grp["SAF_Share_frac"] * grp["Scenario_Prob"]).sum()
    exp_emissions = (grp["CO2_Emissions_Mt"] * grp["Scenario_Prob"]).sum()
    exp_avoided = (grp["Avoided_CO2_Mt"] * grp["Scenario_Prob"]).sum()

    rows.append({
        "Year": year,
        "Expected_Total_Fuel_Mt": total_fuel,
        "Expected_SAF_Share_frac": exp_saf_share,
        "Expected_CO2_Emissions_Mt": exp_emissions,
        "Expected_Avoided_CO2_Mt": exp_avoided
    })

df_prob_weighted = pd.DataFrame(rows).sort_values("Year")
df_prob_weighted["Expected_SAF_Share_pct"] = df_prob_weighted["Expected_SAF_Share_frac"] * 100

print("Probability-weighted expected outcome:")
display(df_prob_weighted.head())
display(df_prob_weighted.tail())

df_prob_weighted.to_csv("ie_iata_prob_weighted_results.csv", index=False)


Probability-weighted expected outcome:


,Year,Expected_Total_Fuel_Mt,Expected_SAF_Share_frac,Expected_CO2_Emissions_Mt,Expected_Avoided_CO2_Mt,Expected_SAF_Share_pct
0,2025,39.624000,0.02000,122.943366,1.872234,2.000
1,2026,40.257984,0.03144,123.822407,2.990242,3.144
2,2027,40.902112,0.04288,124.698104,4.143548,4.288
3,2028,41.556546,0.05432,125.570125,5.332993,5.432
4,2029,42.221450,0.06576,126.438128,6.559440,6.576


,Year,Expected_Total_Fuel_Mt,Expected_SAF_Share_frac,Expected_CO2_Emissions_Mt,Expected_Avoided_CO2_Mt,Expected_SAF_Share_pct
21,2046,55.300134,0.37384,125.354509,48.840912,37.384
22,2047,56.184936,0.39288,124.832870,52.149678,39.288
23,2048,57.083895,0.41192,124.262449,55.551820,41.192
24,2049,57.997237,0.43096,123.641816,59.049481,43.096
25,2050,58.925193,0.45000,122.969512,62.644846,45.000


## 6. Cost model and sensitivity analysis

To address the economic and policy dimension (Objective 2), we implement a simple yet informative cost model.

### Cost model assumptions

- **Base fossil jet fuel price**: `P_jet_base` (€/t).  
  We use a plausible mid-range value (e.g. ~800 €/t), consistent with recent jet fuel prices.
- **SAF price premium** (`spread`): SAF is currently several times more expensive than fossil jet fuel.
  - Literature (IATA, ICCT, ICAO) often cites SAF being **2–5×** the cost of fossil jet, and early e-fuels (PtL) can be even higher.
  - In €/t terms, this translates into a **premium range of roughly 600–1200 €/t** over fossil jet fuel.
- **EU ETS price** (`ETS_price`):  
  Recent and projected EU ETS prices typically range from **50–150 €/tCO₂**, consistent with Phase IV tightening under the EU’s Fit-for-55 package.
- **SAF lifecycle reduction** (`SAF_LCA`):  
  Certified SAF pathways generally deliver **60–90% CO₂ reduction** versus fossil jet, depending on feedstock and technology.

We assume:

- Fossil jet fuel **pays the ETS price** (increasing its effective cost).
- SAF is **exempt from ETS** (conservative and aligned with the idea of incentivising low-carbon fuels).

### Sensitivity grid

We run a 3D sensitivity grid over:

- **SAF price premium (`spread`)**: 600, 900, 1200 €/t  
- **ETS price (`ETS_price`)**: 50, 100, 150 €/tCO₂  
- **SAF lifecycle reduction (`SAF_LCA`)**: 0.6, 0.75, 0.9  (60%, 75%, 90%)

For each combination and milestone year (**2030, 2040, 2050**) in the **Policy scenario (S1)** we compute:

- Extra system cost (€/year) vs a counterfactual with 0% SAF but paying ETS.
- Avoided CO₂ (Mt) using the chosen lifecycle reduction.
- **Marginal abatement cost (MAC)**: €/tCO₂ avoided.

### Why we do this

- **SAF premium** tells us how important technology learning and scaling are.
- **ETS price** captures the role of carbon pricing in closing the cost gap between fossil and SAF.
- **SAF lifecycle performance** captures the difference between “average” and “high-quality” SAF pathways.

Together, these sensitivities allow us to discuss **trade-offs and policy levers** clearly:

- Higher ETS prices → lower abatement costs for SAF.  
- Lower SAF premiums → cheaper climate action.  
- Better SAF LCA (e.g. e-fuels) → more CO₂ avoided per euro spent.


In [ ]:
# ============================================
# Cost model & sensitivity analysis
# ============================================

def run_cost_sensitivity(
    df_saf,
    P_jet_base=800,     # €/t jet fuel
    spread=900,         # SAF premium €/t
    ETS_price=100,      # €/tCO2
    SAF_LCA=0.75,       # fraction reduction
    EF_JET=3.15         # tCO2 / t fuel
):
    df = df_saf.copy()

    EF_SAF = EF_JET * (1 - SAF_LCA)

    P_SAF_base = P_jet_base + spread
    P_jet_eff = P_jet_base + EF_JET * ETS_price    # fossil pays ETS
    P_SAF_eff = P_SAF_base                         # assume SAF exempt

    df["Fuel_jet_Mt"] = df["Total_Fuel_Mt"] * (1 - df["SAF_Share_frac"])
    df["Fuel_SAF_Mt"] = df["Total_Fuel_Mt"] * df["SAF_Share_frac"]

    df["Cost_baseline"] = df["Total_Fuel_Mt"] * 1e6 * P_jet_eff

    df["Cost_with_SAF"] = (
        df["Fuel_jet_Mt"] * 1e6 * P_jet_eff +
        df["Fuel_SAF_Mt"] * 1e6 * P_SAF_eff
    )

    df["Extra_cost"] = df["Cost_with_SAF"] - df["Cost_baseline"]

    df["CO2_Emissions_Mt_LCA"] = df["Total_Fuel_Mt"] * (
        (1 - df["SAF_Share_frac"]) * EF_JET + df["SAF_Share_frac"] * EF_SAF
    )
    df["Avoided_CO2_Mt_LCA"] = df["CO2_Baseline_no_SAF_Mt"] - df["CO2_Emissions_Mt_LCA"]

    df["Cost_per_tCO2"] = np.where(
        df["Avoided_CO2_Mt_LCA"] > 0,
        df["Extra_cost"] / (df["Avoided_CO2_Mt_LCA"] * 1e6),
        np.nan
    )

    df["P_jet_eff"] = P_jet_eff
    df["P_SAF_eff"] = P_SAF_eff
    df["SAF_LCA"] = SAF_LCA
    df["ETS_price"] = ETS_price
    df["spread"] = spread

    return df


In [10]:
# Parameter grids
spreads = [600, 900, 1200]         # €/t SAF premium
ets_prices = [50, 100, 150]        # €/tCO2
saf_lcas = [0.6, 0.75, 0.9]        # 60%, 75%, 90% reduction

sensitivity_rows = []
df_s1 = df_saf_model[df_saf_model["Scenario"] == 1].copy()

for spread in spreads:
    for ets in ets_prices:
        for lca in saf_lcas:
            df_sens = run_cost_sensitivity(
                df_s1,
                P_jet_base=800,
                spread=spread,
                ETS_price=ets,
                SAF_LCA=lca
            )
            # Milestone years
            for target_year in [2030, 2040, 2050]:
                df_y = df_sens[df_sens["Year"] == target_year]
                if df_y.empty:
                    continue
                row = df_y.iloc[0]

                sensitivity_rows.append({
                    "Year": target_year,
                    "spread_€/t": spread,
                    "ETS_price_€/tCO2": ets,
                    "SAF_LCA_reduction": lca,
                    "SAF_Share_%": row["SAF_Share_frac"] * 100,
                    "Extra_cost_€": row["Extra_cost"],
                    "Avoided_CO2_Mt_LCA": row["Avoided_CO2_Mt_LCA"],
                    "Cost_per_tCO2_€/t": row["Cost_per_tCO2"]
                })

df_sensitivity = (
    pd.DataFrame(sensitivity_rows)
    .sort_values(["Year", "spread_€/t", "ETS_price_€/tCO2", "SAF_LCA_reduction"])
    .reset_index(drop=True)
)

print("Sensitivity results (sample):")
display(df_sensitivity.head(20))
display(df_sensitivity.tail(20))

df_sensitivity.to_csv("ie_iata_sensitivity_results.csv", index=False)


Sensitivity results (sample):


,Year,spread_€/t,ETS_price_€/tCO2,SAF_LCA_reduction,SAF_Share_%,Extra_cost_€,Avoided_CO2_Mt_LCA,Cost_per_tCO2_€/t
0,2030,600,50,0.60,8.0,1.518554e+09,6.486025,234.126984
1,2030,600,50,0.75,8.0,1.518554e+09,8.107532,187.301587
2,2030,600,50,0.90,8.0,1.518554e+09,9.729038,156.084656
3,2030,600,100,0.60,8.0,9.780515e+08,6.486025,150.793651
4,2030,600,100,0.75,8.0,9.780515e+08,8.107532,120.634921
5,2030,600,100,0.90,8.0,9.780515e+08,9.729038,100.529101
6,2030,600,150,0.60,8.0,4.375493e+08,6.486025,67.460317
7,2030,600,150,0.75,8.0,4.375493e+08,8.107532,53.968254
8,2030,600,150,0.90,8.0,4.375493e+08,9.729038,44.973545
9,2030,900,50,0.60,8.0,2.548081e+09,6.486025,392.857143


,Year,spread_€/t,ETS_price_€/tCO2,SAF_LCA_reduction,SAF_Share_%,Extra_cost_€,Avoided_CO2_Mt_LCA,Cost_per_tCO2_€/t
61,2050,600,150,0.75,80.0,6.010370e+09,111.368615,53.968254
62,2050,600,150,0.90,80.0,6.010370e+09,133.642338,44.973545
63,2050,900,50,0.60,80.0,3.500156e+10,89.094892,392.857143
64,2050,900,50,0.75,80.0,3.500156e+10,111.368615,314.285714
65,2050,900,50,0.90,80.0,3.500156e+10,133.642338,261.904762
66,2050,900,100,0.60,80.0,2.757699e+10,89.094892,309.523810
67,2050,900,100,0.75,80.0,2.757699e+10,111.368615,247.619048
68,2050,900,100,0.90,80.0,2.757699e+10,133.642338,206.349206
69,2050,900,150,0.60,80.0,2.015242e+10,89.094892,226.190476
70,2050,900,150,0.75,80.0,2.015242e+10,111.368615,180.952381
